In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

**`Load Data`**
+ `one-hot` format

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print('X_train.shape :', mnist.train.images.shape)
print('Y_train.shape :', mnist.train.labels.shape)

X_train.shape : (55000, 784)
Y_train.shape : (55000, 10)


**`mini batch gradient descent`**

In [4]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  # 整除

**`优化程序`**
+ batch_size
+ Neural Network Struct
+ Active Function
+ Cost Function : Cross Entropy
+ 使用其他的优化方式

**`Neural Network Struct`**
+ input  : 784
+ output : 10

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

**`Dropout Param`**

In [6]:
keep_prob = tf.placeholder(tf.float32)

**`tf.nn.softmax`**()

In [7]:
Weight_L1 = tf.Variable(tf.truncated_normal([784,2000], stddev=0.1))  # (input, output)
Biases_L1 = tf.Variable(tf.zeros([2000]) + 0.1)
Z_L1 = tf.matmul(x, Weight_L1) + Biases_L1
A_L1 = tf.nn.tanh(Z_L1)

Drop_L1 = tf.nn.dropout(A_L1, keep_prob)

In [8]:
Weight_L2 = tf.Variable(tf.truncated_normal([2000,2000], stddev=0.1))  # (input, output)
Biases_L2 = tf.Variable(tf.zeros([2000]) + 0.1)
Z_L2 = tf.matmul(Drop_L1, Weight_L2) + Biases_L2
A_L2 = tf.nn.tanh(Z_L2)

Drop_L2 = tf.nn.dropout(A_L2, keep_prob)

In [9]:
Weight_L3 = tf.Variable(tf.truncated_normal([2000,1000], stddev=0.1))  # (input, output)
Biases_L3 = tf.Variable(tf.zeros([1000]) + 0.1)
Z_L3 = tf.matmul(Drop_L2, Weight_L3) + Biases_L3
A_L3 = tf.nn.tanh(Z_L3)

Drop_L3 = tf.nn.dropout(A_L3, keep_prob)

In [10]:
Weight_L4 = tf.Variable(tf.truncated_normal([1000,10], stddev=0.1))  # (input, output)
Biases_L4 = tf.Variable(tf.zeros([10]) + 0.1)
Z_L4 = tf.matmul(Drop_L3, Weight_L4) + Biases_L4
A_L4 = tf.nn.softmax(Z_L4)

Prediction = tf.nn.softmax(Z_L4)

**`tf.nn.softmax_cross_entropy_with_logits`**()

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=Prediction))
#loss = tf.reduce_mean(tf.square(y - Prediction))

lr = tf.Variable(0.3)
optimizer = tf.train.GradientDescentOptimizer(lr)
train = optimizer.minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



**`accuary`**
+ **tf.argmax**(input, axis=None) : Returns the index with the largest value across axes of a tensor
+ **tf.equal**(x, y, name=None) : Returns the truth value of (x == y) element-wise.
+ **tf.cast**(x, dtype) : Casts a tensor to a new type. / 类型转换
+ **tf.reduce_mean**(input_tensor, axis=None) : Computes the mean of elements across dimensions of a tensor.

In [14]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Prediction, 1))
    # return a bool-list

accuary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**由于该神经网络过于复杂，容易出现overfitting**
+ 因此使用dropout来避免过拟合
+ 当使用dropout之后
    + test accuracy的上升速度变慢，[ 从 96 -> 97 ]， 收敛速度变慢
    + **train accuracy的准确率没有很高**，**test & train accuracy 相对偏差不是很大**

In [13]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 1.0
            })
        #if epoch % 100 == 0:
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.9517  Train Acuuary 0.9631818
Iter 1  Test Acuuary 0.9587  Train Acuuary 0.9761818
Iter 2  Test Acuuary 0.9671  Train Acuuary 0.98423636
Iter 3  Test Acuuary 0.9698  Train Acuuary 0.9881455
Iter 4  Test Acuuary 0.9721  Train Acuuary 0.99023634
Iter 5  Test Acuuary 0.9718  Train Acuuary 0.9914727
Iter 6  Test Acuuary 0.9713  Train Acuuary 0.99227273
Iter 7  Test Acuuary 0.9724  Train Acuuary 0.993
Iter 8  Test Acuuary 0.9719  Train Acuuary 0.9935455
Iter 9  Test Acuuary 0.9725  Train Acuuary 0.9940182
Iter 10  Test Acuuary 0.9732  Train Acuuary 0.9942727
Iter 11  Test Acuuary 0.973  Train Acuuary 0.9944364
Iter 12  Test Acuuary 0.9728  Train Acuuary 0.9946182
Iter 13  Test Acuuary 0.9726  Train Acuuary 0.9947818
Iter 14  Test Acuuary 0.9733  Train Acuuary 0.99485457
Iter 15  Test Acuuary 0.9734  Train Acuuary 0.995
Iter 16  Test Acuuary 0.9739  Train Acuuary 0.99503636
Iter 17  Test Acuuary 0.9739  Train Acuuary 0.99521816
Iter 18  Test Acuuary 0.9746  Train Acuua

In [15]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 0.7
            })
        #if epoch % 100 == 0:
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.9242  Train Acuuary 0.9175636
Iter 1  Test Acuuary 0.932  Train Acuuary 0.9312364
Iter 2  Test Acuuary 0.9377  Train Acuuary 0.9381091
Iter 3  Test Acuuary 0.946  Train Acuuary 0.94447273
Iter 4  Test Acuuary 0.9473  Train Acuuary 0.9482727
Iter 5  Test Acuuary 0.951  Train Acuuary 0.9525818
Iter 6  Test Acuuary 0.9539  Train Acuuary 0.9553273
Iter 7  Test Acuuary 0.9554  Train Acuuary 0.9567091
Iter 8  Test Acuuary 0.9575  Train Acuuary 0.9595636
Iter 9  Test Acuuary 0.9603  Train Acuuary 0.96147275
Iter 10  Test Acuuary 0.9617  Train Acuuary 0.9632909
Iter 11  Test Acuuary 0.9609  Train Acuuary 0.9634
Iter 12  Test Acuuary 0.9607  Train Acuuary 0.96567273
Iter 13  Test Acuuary 0.9628  Train Acuuary 0.9676727
Iter 14  Test Acuuary 0.9647  Train Acuuary 0.969
Iter 15  Test Acuuary 0.9656  Train Acuuary 0.96956366
Iter 16  Test Acuuary 0.9653  Train Acuuary 0.9702909
Iter 17  Test Acuuary 0.9662  Train Acuuary 0.97114545
Iter 18  Test Acuuary 0.9677  Train Acuuary